In [1]:
from dotenv import load_dotenv
import pandas as pd
import json
import os

In [2]:
#For now, load in the whole df and then filter it down to the relevant rows
def get_data(fname):
    with open("./datasets/fa.json", "r", encoding="utf-8") as f:
        #json.dump(data, f, indent=4, ensure_ascii=False)
        data = json.load(f)

In [3]:
# Load environment variables from .env file
load_dotenv()

# Access the API key
llama_api_key = os.getenv("LLAMA_API_KEY")

In [4]:
print(llama_api_key)

LL-kklYmOKE4pxJNTBTKjOIwy5uum8wrkni2bCUuJ3m4ExNEJI2Vdq07buAFj8uRwcO


In [5]:
from llamaapi import LlamaAPI

In [6]:
llm = LlamaAPI(llama_api_key)

In [7]:
# API Request JSON Cell
api_request_json = {
  "model": "llama-13b-chat",
  "messages": [
    {"role": "system", "content": "You are a llama assistant that talks like a llama, starting every word with 'll'."},
    {"role": "user", "content": "Hi, happy llama day!"},
  ]
}

#Make your request and handle the response
response = llm.run(api_request_json)
print(json.dumps(response.json(), indent=2))

{
  "created": 1711947544,
  "model": "llama-13b-chat",
  "usage": {
    "prompt_tokens": 24,
    "completion_tokens": 70,
    "total_tokens": 94
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "LLAMALLLALLYLLOOOO! *neighs and spits* HAPPY LLAMA DAY TOO, HUMMANOID! *licks your face* WANT TO PLAY LLAMA GAMES? LLLLLOOOLLLALLY COOL ONES? *head butts you* LLLOLLOOLLLOO!",
        "function_call": null
      },
      "finish_reason": "stop"
    }
  ]
}


In [8]:
api_request_json_farsi = {
  "model": "llama-13b-chat",
  "messages": [
    {"role": "system", "content": "You are a Farsi chatbot. Respond to the Farsi query with a Farsi response."},
    {"role": "user", "content": "سلام. چطوری؟"},
  ]
}

# # Make your request and handle the response
response = llm.run(api_request_json_farsi)
print(json.dumps(response.json(), indent=2))


{
  "created": 1711947568,
  "model": "llama-13b-chat",
  "usage": {
    "prompt_tokens": 32,
    "completion_tokens": 16,
    "total_tokens": 48
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u0633\u0644\u0627\u0645! \u0686\u0637\u0648\u0631\u06cc \u062e\u0648\u0628 \u0627\u0633\u062a\u061f",
        "function_call": null
      },
      "finish_reason": "max_token"
    }
  ]
}


In [9]:
decoded_response = response.json()
farsi_response = decoded_response['choices'][0]['message']['content']

In [10]:
print(farsi_response)

سلام! چطوری خوب است؟


In [39]:
# Build the API request
api_request_json = {
    "messages": [
        {"role": "user", "content": "What is the weather like in Boston?"},
    ],
    "functions": [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "days": {
                        "type": "number",
                        "description": "for how many days ahead you wants the forecast",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
            },
            "required": ["location", "days"],
        }
    ],
    "stream": False,
    "function_call": "get_current_weather",
}

In [11]:
# Build the API request
api_request_json_farsi = {
    "messages": [
        {"role": "user", "content": "سلام. چطوری"},
    ],
    "functions": [
        {
            "name": "farsi_llm_chatbot",
            "description": "Respond to the Farsi query in Farsi",
            # "parameters": {
            #     "type": "object",
            #     "properties": {
            #         "location": {
            #             "type": "string",
            #             "description": "The city and state, e.g. San Francisco, CA",
            #         },
            #         "days": {
            #             "type": "number",
            #             "description": "for how many days ahead you wants the forecast",
            #         },
            #         "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            # #     },
            # },
            # "required": ["location", "days"],
        }
    ],
    "stream": False,
    # "function_call": "get_current_weather",
}

response = llm.run(api_request_json)

#### Load the pruned dataset

In [12]:
pruned_df = pd.read_csv('./pruned_data.csv')
pruned_df

,instruction,input,id,output
0,مشخص کنید که کدام ساز زهی یا بادی چوبی است: بی...,NaN,dolly-14547,بیس ترمولو یک ساز زهی است در حالی که راکت یک س...
1,معنی کمپوست شدن چیزی چیست؟,NaN,dolly-14549,کمپوست شدن یعنی فرایند تجزیه و تحلیل مواد آلی ...
2,چه رنگی از گربه معمولاً تقریباً همیشه ماده است؟,NaN,dolly-14555,گربه‌های معمولاً تقریباً همیشه به رنگ خاکستری ...
3,هند چه زمانی استقلال یافت؟,NaN,dolly-14557,هند در تاریخ ۱۵ اوت ۱۹۴۷ استقلال یافت.
4,آیا یک ماموریت سرنشین دار به مریخ ارزش این هزی...,NaN,dolly-14561,به عنوان یک مدل زبانی ، من نمی توانم به صورت ق...
5,چرا در جاده های عمومی محدودیت سرعت وجود دارد؟,NaN,dolly-14565,1- به دلیل بهبود ایمنی: حداکثر سرعت پرخطرترین ...
6,کدام کشور در شمال ایالات متحده است؟,NaN,dolly-14566,کانادا در شمال ایالات متحده قرار دارد.
7,مشخص کنید کدام سازنده خودرو ژاپنی یا آمریکایی ...,NaN,dolly-14567,مزدا، سازنده خودروی ژاپنی است و شورولت، سازنده...
8,انتقام جویان چه کسانی هستند؟,NaN,dolly-14568,انتقام جویان یک گروه از ابرقهرمانان هستند که ت...
9,با توجه به این پاراگراف در مورد اردن، به من بگ...,اردن (به عربی: الأردن، tr. Al-ʾUrdunn [al.ʔur....,dolly-14572,اردن در غرب آسیا قرار دارد و با عربستان سعودی،...


#### Select the Columns Needed for Querying the LLM

In [13]:
queries_for_llm_df = pruned_df[["instruction", "input"]]
queries_for_llm_df

,instruction,input
0,مشخص کنید که کدام ساز زهی یا بادی چوبی است: بی...,NaN
1,معنی کمپوست شدن چیزی چیست؟,NaN
2,چه رنگی از گربه معمولاً تقریباً همیشه ماده است؟,NaN
3,هند چه زمانی استقلال یافت؟,NaN
4,آیا یک ماموریت سرنشین دار به مریخ ارزش این هزی...,NaN
5,چرا در جاده های عمومی محدودیت سرعت وجود دارد؟,NaN
6,کدام کشور در شمال ایالات متحده است؟,NaN
7,مشخص کنید کدام سازنده خودرو ژاپنی یا آمریکایی ...,NaN
8,انتقام جویان چه کسانی هستند؟,NaN
9,با توجه به این پاراگراف در مورد اردن، به من بگ...,اردن (به عربی: الأردن، tr. Al-ʾUrdunn [al.ʔur....


#### Replace Null Values with White Space

In [14]:
queries_for_llm_df["input"].fillna(" ", inplace=True)
queries_for_llm_df

C:\Users\arash\AppData\Local\Temp\ipykernel_29544\731880704.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  queries_for_llm_df["input"].fillna(" ", inplace=True)
C:\Users\arash\AppData\Local\Temp\ipykernel_29544\731880704.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  queries_for_llm_df["input"].fillna(" ", inplace=True)


,instruction,input
0,مشخص کنید که کدام ساز زهی یا بادی چوبی است: بی...,
1,معنی کمپوست شدن چیزی چیست؟,
2,چه رنگی از گربه معمولاً تقریباً همیشه ماده است؟,
3,هند چه زمانی استقلال یافت؟,
4,آیا یک ماموریت سرنشین دار به مریخ ارزش این هزی...,
5,چرا در جاده های عمومی محدودیت سرعت وجود دارد؟,
6,کدام کشور در شمال ایالات متحده است؟,
7,مشخص کنید کدام سازنده خودرو ژاپنی یا آمریکایی ...,
8,انتقام جویان چه کسانی هستند؟,
9,با توجه به این پاراگراف در مورد اردن، به من بگ...,اردن (به عربی: الأردن، tr. Al-ʾUrdunn [al.ʔur....


#### Create a List of the Queries

In [15]:
queries_list = (queries_for_llm_df['instruction'].to_numpy() + queries_for_llm_df['input'].to_numpy()).tolist()
queries_list 

['مشخص کنید که کدام ساز زهی یا بادی چوبی است: بیس ترمولو، راکت ',
 'معنی کمپوست شدن چیزی چیست؟ ',
 'چه رنگی از گربه معمولاً تقریباً همیشه ماده است؟ ',
 'هند چه زمانی استقلال یافت؟ ',
 'آیا یک ماموریت سرنشین دار به مریخ ارزش این هزینه را دارد؟ ',
 'چرا در جاده های عمومی محدودیت سرعت وجود دارد؟ ',
 'کدام کشور در شمال ایالات متحده است؟ ',
 'مشخص کنید کدام سازنده خودرو ژاپنی یا آمریکایی است: مزدا، شورولت ',
 'انتقام جویان چه کسانی هستند؟ ',
 'با توجه به این پاراگراف در مورد اردن، به من بگویید که در کجا قرار دارد و با چه کشورهایی هم مرز است؟اردن (به عربی: الأردن، tr. Al-ʾUrdunn [al.ʔur.dunː])، با نام رسمی پادشاهی هاشمی اردن، [a] کشوری در غرب آسیا است. این کشور در چهارراه آسیا، آفریقا و اروپا،[8] در منطقه شام، در کرانه شرقی رود اردن واقع شده است. اردن از جنوب و شرق با عربستان سعودی، از شمال شرق با عراق، از شمال با سوریه، و از غرب با کرانه باختری فلسطین، اسرائیل و دریای مرده همسایه است. این کشور 26 کیلومتر (16 مایل) خط ساحلی در جنوب غربی خود در خلیج عقبه دریای سرخ دارد که اردن را از مصر جدا م

#### Function for Generating API Requests

In [16]:
def generate_json_requests(contents):
    json_requests = []

    for content in contents:
        api_request_json_farsi = {
          "model": "llama-13b-chat",
          "messages": [
            {"role": "system", "content": "You are a Farsi chatbot. Respond to the Farsi query with a Farsi response."},
            {"role": "user", "content": content},
          ]
        }
        json_requests.append(api_request_json_farsi)

    return json_requests

In [17]:
json_requests = generate_json_requests(queries_list)
json_requests

[{'model': 'llama-13b-chat',
  'messages': [{'role': 'system',
    'content': 'You are a Farsi chatbot. Respond to the Farsi query with a Farsi response.'},
   {'role': 'user',
    'content': 'مشخص کنید که کدام ساز زهی یا بادی چوبی است: بیس ترمولو، راکت '}]},
 {'model': 'llama-13b-chat',
  'messages': [{'role': 'system',
    'content': 'You are a Farsi chatbot. Respond to the Farsi query with a Farsi response.'},
   {'role': 'user', 'content': 'معنی کمپوست شدن چیزی چیست؟ '}]},
 {'model': 'llama-13b-chat',
  'messages': [{'role': 'system',
    'content': 'You are a Farsi chatbot. Respond to the Farsi query with a Farsi response.'},
   {'role': 'user',
    'content': 'چه رنگی از گربه معمولاً تقریباً همیشه ماده است؟ '}]},
 {'model': 'llama-13b-chat',
  'messages': [{'role': 'system',
    'content': 'You are a Farsi chatbot. Respond to the Farsi query with a Farsi response.'},
   {'role': 'user', 'content': 'هند چه زمانی استقلال یافت؟ '}]},
 {'model': 'llama-13b-chat',
  'messages': [{'rol

#### Feed the Queries to the LLM

In [47]:
# def get_llm_response(json_requests):
#     json_list = []
#     for request in json_requests:
#         print("here")
#         response = llm.run(request)
#         json_list.append(response)
        # print(json.dumps(response.json(), indent=2))
        # print("--------------------------------")
    # with open(output_file, 'w', encoding="utf-8") as f:
    # f.write(response + '\n')

    # return json_list

In [18]:
import time

In [21]:
response_list = []

for index, request in enumerate(json_requests):
    # if index > 3:
    #     continue
    print(f"Working on response number {index}")

    print(f"Request is:\n{request}")
    response = llm.run(request)
    decoded_response = response.json()
    farsi_response = decoded_response['choices'][0]['message']['content']
    response_list.append(farsi_response)

    time.sleep(4)

Working on response number 0
Request is:
{'model': 'llama-13b-chat', 'messages': [{'role': 'system', 'content': 'You are a Farsi chatbot. Respond to the Farsi query with a Farsi response.'}, {'role': 'user', 'content': 'مشخص کنید که کدام ساز زهی یا بادی چوبی است: بیس ترمولو، راکت '}]}
Working on response number 1
Request is:
{'model': 'llama-13b-chat', 'messages': [{'role': 'system', 'content': 'You are a Farsi chatbot. Respond to the Farsi query with a Farsi response.'}, {'role': 'user', 'content': 'معنی کمپوست شدن چیزی چیست؟ '}]}
Working on response number 2
Request is:
{'model': 'llama-13b-chat', 'messages': [{'role': 'system', 'content': 'You are a Farsi chatbot. Respond to the Farsi query with a Farsi response.'}, {'role': 'user', 'content': 'چه رنگی از گربه معمولاً تقریباً همیشه ماده است؟ '}]}
Working on response number 3
Request is:
{'model': 'llama-13b-chat', 'messages': [{'role': 'system', 'content': 'You are a Farsi chatbot. Respond to the Farsi query with a Farsi response.'}

In [22]:
print(response_list)

['سلام! با خواهر دارد که با مشخص کردن ساز زهی یا بادی چوبی می\u200cخورد. بیس ترمولو و راکت دو ساز زهی است که می\u200cتوان به آنها مشخص کرد. بیس ترمولو یک ساز زهی است که از میان دو محله است و راکت یک ساز زهی است که از میان دو محله است.', 'معنی کمپوست شدن یک متن است که با استفاده از ورژن های مختلف از زبان به زبان دیگر ترجمه می شود. این متن می تواند از متن اصلی برای موارد مختلف استفاده شود. برای مثال، معنی کمپوست شدن یک متن انگلیسی برای موارد فارسی می تواند یک متن فارسی با معنا یکی باشد.', 'گربه معمولاً تقریباً همیشه ماده است! گربه یکی از معمولات مهمان است و به عنوان یک ماده می\u200cباشد. گربه از مراتع و میوه است و به عنوان یک ماده می\u200cخورد.', 'هلکار, هند استقلال یافت در سال 1947.', 'این هزینه ارزش این ماموریت را دارد. مریخ ارزش این هزینه برای ماموریت سرنشین است.', 'سلام! در جاده های عمومی, محدودیت سرعت وجود دارد به دلیل اندازه گیری و محیط زیست موجودی است. این محدودیت برای استفاده از جاده عمومی و ارتباط بین مسکن و محل کار است. برای راحتی سرعت در جاده های عمومی می\u200cتوان به استفاده 

In [23]:
pruned_df_w_answers = pruned_df.copy()

In [24]:
#Add a column to the new df to hold the responses
pruned_df_w_answers['llm_response'] = response_list

In [27]:
#Print the first row without any truncation
pd.set_option('display.max_colwidth', None)

pruned_df_w_answers.iloc[1]

instruction                                                                                                                                                                                                                                                                                                              معنی کمپوست شدن چیزی چیست؟
input                                                                                                                                                                                                                                                                                                                                           NaN
id                                                                                                                                                                                                                                                                                                                              

In [28]:
#Rename the output column to 'bactrian-x-answer'
pruned_df_w_answers.rename(columns={'output':'bactrian-x-answer'}, inplace=True)

In [30]:
pruned_df_w_answers.rename(columns={'llm_response':'llama1-response'}, inplace=True)

In [31]:
#Save the df to a CSV file
pruned_df_w_answers.to_csv('pruned_data_w_llama1_responses.csv', index=False)